In [ ]:
# pip install pytorchvideo
# pip install pytorchvideo-nightly

In [ ]:
import sys
sys.path.append('pytorchvideo')
from pytorchvideo.data import LabeledVideoDataset

In [ ]:
from glob import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# I have those csv files in my directory already splitted.
train_df = pd.read_csv('../data_temporal/train1.csv')
val_df = pd.read_csv('../data_temporal/val1.csv')

In [ ]:
train_df.head()

In [ ]:
val_df.head()

In [ ]:
print(f"Training dataset: {len(train_df)}, Validation dataset {len(val_df)}")

# Augmentation Process

In [ ]:
# Augmentation process
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset


from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,

    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

In [ ]:
video_transform = Compose([
    ApplyTransformToKey(key="video",    
    transform = Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x:x/255),
        NormalizeVideo(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
        RandomShortSideScale(min_size=248, max_size=256),
        CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5)
    ]),
    ),
])

In [ ]:
from torch.utils.data import DataLoader

# Labeled video dataset
train_dataset = labeled_video_dataset(train_df,
                                      clip_sampler=make_clip_sampler('random', 2),
                                        transform = video_transform, decode_audio=False)


loader = DataLoader(train_dataset, batch_size=2, num_workers=0, pin_memory=True)

In [ ]:
batch = next(iter(loader))

In [ ]:
batch.keys()

In [ ]:
batch['video'].shape, batch['label'].shape
